In [11]:
import findspark
findspark.init()
#1a01e0e77d803e8e292dbb639aa77646d41947892fe6f227

In [12]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Files") \
    .master("spark://f5db43ce3d38:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

25/04/03 01:38:07 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
from grandeInformacion.spark_utils import SparkUtils

logs = spark \
                .readStream \
                .format("text") \
                .option("cleanSource", "archive") \
                .option("sourceArchiveDir", "/home/jovyan/archive") \
                .load("/home/jovyan/notebooks/data/logs_output/")

-------------------------------------------
Batch: 4
-------------------------------------------
+-----+
|value|
+-----+
+-----+



In [ ]:
from pyspark.sql.functions import split

logs_df = logs.select(split(logs.value, " ").alias("log_array"))
logs_df = logs_df.withColumn("date", logs_df["log_array"].getItem(0).cast("date"))
logs_df = logs_df.withColumn("file", logs_df["log_array"].getItem(0).cast("date"))


logs_df.printSchema()

In [ ]:
query = logs_df \
                .writeStream \
                .outputMode("append") \
                .trigger(processingTime='20 seconds') \
                .format("console") \
                .start()

query.awaitTermination()

25/04/03 00:51:58 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-92d6cc51-394a-40a9-b457-25ab9ecf61a0. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/03 00:51:58 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------+
|               value|
+--------------------+
|1743641481.543713...|
|1743641486.543907...|
|1743641491.544776...|
|1743641496.545192...|
|1743641501.545385...|
+--------------------+

-------------------------------------------
Batch: 5
-------------------------------------------
+-----+
|value|
+-----+
+-----+



25/04/03 00:52:40 WARN FileStreamSource$SourceFileArchiver: Fail to move file:/home/jovyan/notebooks/data/logs_output/log-1743641481.535084.txt to file:/home/jovyan/archive/home/jovyan/notebooks/data/logs_output/log-1743641481.535084.txt / skip moving file.
25/04/03 00:52:40 WARN FileStreamSource$SourceFileArchiver: Fail to move file:/home/jovyan/notebooks/data/logs_output/log-1743641511.5483212.txt to file:/home/jovyan/archive/home/jovyan/notebooks/data/logs_output/log-1743641511.5483212.txt / skip moving file.


-------------------------------------------
Batch: 1
-------------------------------------------
-------------------------------------------
Batch: 6
-------------------------------------------
+-----+
|value|
+-----+
+-----+

+-----+
|value|
+-----+
+-----+



25/04/03 00:53:00 WARN FileStreamSource$SourceFileArchiver: Fail to move file:/home/jovyan/notebooks/data/logs_output/log-1743641541.545763.txt to file:/home/jovyan/archive/home/jovyan/notebooks/data/logs_output/log-1743641541.545763.txt / skip moving file.


-------------------------------------------
Batch: 7
-------------------------------------------
-------------------------------------------
Batch: 2
-------------------------------------------
+-----+
|value|
+-----+
+-----+

+-----+
|value|
+-----+
+-----+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/spark-3.5.4-bin-hadoop3-scala2.13/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

25/04/03 00:53:40 WARN FileStreamSource$SourceFileArchiver: Fail to move file:/home/jovyan/notebooks/data/logs_output/log-1743641571.5544786.txt to file:/home/jovyan/archive/home/jovyan/notebooks/data/logs_output/log-1743641571.5544786.txt / skip moving file.


-------------------------------------------
Batch: 3
-------------------------------------------
+-----+
|value|
+-----+
+-----+

-------------------------------------------
Batch: 8
-------------------------------------------
+-----+
|value|
+-----+
+-----+



In [ ]:
query.stop()

In [4]:
sc.stop()